In [ ]:
"""
  - use imgaug.yml
"""

In [ ]:
"""
Trick 1: Rescale bounding box together with image
- json is a yolo json label file created with tools like makesense.ai, etc
"""
import imgaug as ia
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import cv2
import json
import numpy as np

def read_json(path):
    minx = min (np.array(coordinate)[:,0])
    maxx = max (np.array(coordinate)[:,0])
    miny = min(np.array(coordinate)[:,1])
    maxy = max(np.array(coordinate)[:,1])
    return minx, maxx, miny, maxy

img_path ="/home/thule/Project/document_split1/Aircraft IT MRO v2 i1 WP ADS Records (3) image 6 viewer.png"
img = cv2.imread(img_path)
json_path="/home/thule/Project/aircraft/Aircraft IT MRO v2 i1 WP ADS Records (3) image 6 viewer.json"
label_matrix= ["type", "write", "signature", "mark"]

bbs=[]
f=open(json_path, "r")
data = json.loads(f.read())
# Convert to Yolo
image_height=data['imageHeight']
image_width = data['imageWidth']

for x in (data['shapes']):
    coordinate = x['points']
    label = x['label']
    if (label_matrix.count(label)>0):
        # Prepare text file
        minx, maxx, miny, maxy = read_json(coordinate)
        box = BoundingBox(x1=minx, x2= maxx,y1= miny, y2=maxy, label=label)
        bbs.append(box)

bbs_oi = BoundingBoxesOnImage(bbs, shape=(image_height, image_width))

# Rescale image and bounding boxes
# image_rescaled = ia.imresize_single_image(img, ( image_height*2, image_width*2))
image_rescaled = ia.imresize_single_image(img, ( image_width*2, image_height*2))
bbs_rescaled = bbs_oi.on(image_rescaled)
print(bbs_rescaled[0][0][0])
print(bbs_rescaled[0][1])
# Draw image before/after rescaling and with rescaled bounding boxes
image_bbs = bbs_oi.draw_on_image(img, size=2)
image_rescaled_bbs = bbs_rescaled.draw_on_image(image_rescaled, size=2)
cv2.imwrite( "ori.jpg", image_bbs)
cv2.imwrite("scal.jpg", image_rescaled_bbs)